# Noise transformation
The purpose of this notebook is to empirically confirm the theoretical formulae on the transformation of noise passing from polar to Cartesian coordinates.

To perform the check, we proceed as follows:

1. we assume to measure the voltage on two nodes on 100,000 different time instants,
2. we then add Gaussian noise to the measurements in polar coordinates,
3. we transform the noisy measurement into Cartesian coordinates,
4. finally, we extract the noise in Cartesian coordinate and we normalize it using its theoretical variance. 

We want to see that the normalized distribution is a standard Gaussian.

# Setup

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [ ]:
sys.path.append('..')
%load_ext autoreload
%autoreload 2

In [ ]:
from src.simulation.noise import add_polar_noise_to_measurement

In [ ]:
samples = 100000
nodes = 2
v_range = np.array([0.8, 1])
theta_range = np.array([-20, 0]) / 180 * np.pi
v_sd = 0.02
theta_sd = 0.05 * np.pi / 180

# Ground truth

In [ ]:
np.random.seed(11)

v = np.random.uniform(low=v_range[0], high=v_range[1], size=(samples, nodes))
theta = np.random.uniform(low=theta_range[0], high=theta_range[1], size=(samples, nodes))

noise_free_values = v * np.exp(1j * theta)

# Add noise in polar coordinates
We add noise in polar coordinate and we transform back to Cartesian. Then, we extract the noise by exploiting the knowledge of the real, noise-free values. 

In [ ]:
np.random.seed(11)

v_noise = np.random.normal(0, v_sd, v.shape)
theta_noise = np.random.normal(0, theta_sd, theta.shape)
noisy_v = v + v_noise
noisy_theta = theta + theta_noise
noisy_measurement = noisy_v * np.exp(1j * noisy_theta)

noise = noisy_measurement - noise_free_values

# Check noise distribution
At first, we naively try to check the distribution of the noise in Cartesian coordinates without any further transformation.

In [ ]:
def plot_distribution(real_data, imag_data):
    fig, ax = plt.subplots(2, 2, sharex=True, sharey=True)
    fig.suptitle('QQ plots')
    ax[0, 0].set_title('Re(V_1)')
    sm.qqplot(real_data[:, 0], line='45', ax=ax[0, 0])
    ax[0, 1].set_title('Re(V_2)')
    sm.qqplot(real_data[:, 1], line='45', ax=ax[0, 1])
    ax[1, 0].set_title('Im(V_1)')
    sm.qqplot(imag_data[:, 0], line='45', ax=ax[1, 0])
    ax[1, 1].set_title('Im(V_2)')
    sm.qqplot(imag_data[:, 1], line='45', ax=ax[1, 1])
    plt.tight_layout()
    plt.show()

    fig, ax = plt.subplots(2, 2, sharex=True, sharey=True)
    fig.suptitle('Empirical distribution')
    ax[0, 0].set_title('Re(V_1)')
    ax[0, 0].hist(real_data[:, 0], bins=50)
    ax[0, 1].set_title('Re(V_2)')
    ax[0, 1].hist(real_data[:, 1], bins=50)
    ax[1, 0].set_title('Im(V_1)')
    ax[1, 0].hist(imag_data[:, 0], bins=50)
    ax[1, 1].set_title('Im(V_2)')
    ax[1, 1].hist(imag_data[:, 1], bins=50)
    plt.show()

In [ ]:
plot_distribution(np.real(noise), np.imag(noise))

We should remember that the noise in Cartesian coordinates is not IID. In fact, the variance depends on the real value of each sample. 

Thus, in order to check for the noise distribution, we need to normalize each sample for its standard deviation and then check if the distribution of real and imaginary parts are Gaussian.

In [ ]:
v_var = v_sd ** 2
theta_var = theta_sd ** 2

real_var = (v ** 2) * np.exp(-theta_var) * ((np.cos(theta)**2) * (np.cosh(theta_var) - 1) + (np.sin(theta)**2) * np.sinh(theta_var)) + v_var * np.exp(-theta_var) * ((np.cos(theta)**2) * np.cosh(theta_var) + (np.sin(theta)**2) * np.sinh(theta_var))
imag_var = (v ** 2) * np.exp(-theta_var) * ((np.sin(theta)**2) * (np.cosh(theta_var) - 1) + (np.cos(theta)**2) * np.sinh(theta_var)) + v_var * np.exp(-theta_var) * ((np.sin(theta)**2) * np.cosh(theta_var) + (np.cos(theta)**2) * np.sinh(theta_var))

real_sd, imag_sd = np.sqrt(real_var), np.sqrt(imag_var)

real_norm = np.real(noise) / real_sd
imag_norm = np.imag(noise) / imag_sd

In [ ]:
plot_distribution(real_norm, imag_norm)